<a href="https://colab.research.google.com/github/avrilstihler/Recomendacao-de-Livros-com-IA/blob/main/sistema_recomendacao_livros_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Etapa 1: Instalar as bibliotecas necessárias
%pip -q install google-genai google-adk

# Etapa 2: Configurar a API Key do Google Gemini
import os
from google.colab import userdata
import warnings

# Suprimir avisos que podem poluir a saída (opcional)
warnings.filterwarnings("ignore")

try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
except userdata.SecretNotFoundError as e:
    print(f"Secret 'GOOGLE_API_KEY' não encontrado. Por favor, configure-o no Colab Secrets.")
    print("Instruções: https://medium.com/@lucasbm/google-colab-secrets-41014890771a")
    raise e

# Etapa 3: Configurar o cliente da SDK do Gemini e o ADK
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types as genai_types # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab

client = genai.Client()
MODEL_ID = "gemini-2.0-flash" # Ou outro modelo Gemini que preferir

# Etapa 4: Funções Auxiliares (mesmas do original)
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = genai_types.Content(role="user", parts=[genai_types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

def to_markdown(text):
  text = text.replace('•', '  *')
  if text and text.strip():
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))
  return Markdown("> _Nenhuma resposta ou resposta vazia do agente._")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72

In [2]:
#--------------------------------------------------------
# AGENTES PARA DESCOBERTA E RECOMENDAÇÃO LITERÁRIA
#--------------------------------------------------------

###################################################
# --- Agente 1: Detetive Literário (Profiler) --- #
###################################################
def agente_detetive_literario(livro_alvo: str, perfil_do_leitor: str = ""):
    """
    Investiga um livro específico, buscando informações relevantes para um leitor decidir se deve lê-lo
    ou para extrair características chave para recomendações.
    """
    instrucao = f"""
    Você é um Detetive Literário especialista. Sua missão é investigar o livro "{livro_alvo}".
    Compile um perfil conciso, focando em:
    1.  **Sinopse Breve:** Sem spoilers cruciais.
    2.  **Temas Principais:** Quais assuntos o livro explora?
    3.  **Estilo de Escrita e Tom:** É descritivo, rápido, poético, sombrio, engraçado, etc.?
    4.  **Sentimento Geral / Vibe:** Que tipo de emoção ou atmosfera o livro costuma evocar nos leitores? (ex: inspirador, melancólico, eletrizante, reflexivo, divertido).
    5.  **Público-Alvo Comum / Para Quem é Recomendado:** Que tipo de leitor costuma apreciar este livro?
    6.  **Pontos Fortes Comuns:** O que os leitores e críticos geralmente elogiam?
    7.  **Possíveis Pontos Fracos ou Advertências:** Há aspectos que alguns leitores podem não gostar? (ex: ritmo lento, temas pesados, complexidade).

    Se um "perfil do leitor" for fornecido, tente sutilmente relacionar os achados com esse perfil, mas o foco principal é o perfil do livro.
    Use a ferramenta de busca (google_search) para encontrar resenhas, análises, discussões em fóruns, e informações de editoras.
    Seja objetivo e informativo. Formate como um relatório de detetive, com seções claras.
    """
    if perfil_do_leitor:
        instrucao += f"\n\nConsidere também este perfil do leitor ao moldar sua resposta: {perfil_do_leitor}"

    detetive = Agent(
        name="agente_detetive_literario",
        model=MODEL_ID,
        instruction=instrucao,
        description="Agente que cria um perfil detalhado de um livro para avaliação ou recomendação.",
        tools=[google_search]
    )
    entrada_do_agente = f"Investigar o livro: {livro_alvo}"
    perfil_do_livro = call_agent(detetive, entrada_do_agente)
    return perfil_do_livro

#######################################################
# --- Agente 2: Conector de Histórias (Similarity) --- #
#######################################################
def agente_conector_de_historias(referencia: str, tipo_referencia: str, perfil_do_leitor: str = ""):
    """
    Encontra livros similares a uma referência (livro ou tema).
    tipo_referencia pode ser 'livro_base' ou 'tema_interesse'.
    """
    instrucao_base = f"""
    Você é um Conector de Histórias, um especialista em encontrar elos literários.
    Sua tarefa é encontrar 3 a 5 livros que sejam SIMILARES à referência fornecida.
    """

    if tipo_referencia == 'livro_base':
        instrucao = instrucao_base + f"""
        A referência é o livro: "{referencia}".
        Primeiro, use a ferramenta de busca (google_search) para entender rapidamente os principais temas, estilo, tom e sentimento/vibe deste livro de referência, caso não seja óbvio.
        Depois, busque outros livros que compartilhem similaridades significativas em um ou mais dos seguintes aspectos:
        - Temas e Assuntos abordados.
        - Estilo de Escrita e Narrativa.
        - Tom (ex: humorístico, sério, cínico).
        - Sentimento/Vibe que o livro passa (ex: aventura épica, conforto, mistério instigante).
        - Complexidade ou Abordagem.
        """
    elif tipo_referencia == 'tema_interesse':
        instrucao = instrucao_base + f"""
        A referência é o tema de interesse: "{referencia}".
        Use a ferramenta de busca (google_search) para encontrar livros populares e bem avaliados que se encaixem neste tema.
        Tente variar as sugestões em termos de subgênero ou abordagem dentro do tema, se possível.
        """
    else:
        return "Tipo de referência inválido para o Agente Conector de Histórias."

    if perfil_do_leitor:
        instrucao += f"\n\nConsidere este perfil do leitor ao selecionar as similaridades e fazer sugestões: {perfil_do_leitor}"

    instrucao += """
    Para cada livro similar encontrado, forneça:
    - Título e Autor.
    - Uma breve justificativa (1-2 frases) explicando POR QUE ele é similar ou relevante à referência, destacando os pontos de conexão.
    Use a ferramenta de busca (google_search) extensivamente.
    Apresente a lista de forma clara.
    """

    conector = Agent(
        name="agente_conector_de_historias",
        model=MODEL_ID,
        instruction=instrucao,
        description="Agente que encontra livros similares com base em um livro ou tema de referência.",
        tools=[google_search]
    )

    entrada_do_agente = f"Encontrar livros similares para: {referencia} (Tipo: {tipo_referencia})"
    livros_similares = call_agent(conector, entrada_do_agente)
    return livros_similares

###############################################################
# --- Agente 3: Curador de Recomendações (Generator) --- #
###############################################################
def agente_curador_de_recomendacoes(dados_entrada: str, tipo_solicitacao: str, livro_original_ou_tema:str, perfil_do_leitor: str = ""):
    """
    Apresenta as informações de forma útil e como uma recomendação final ao usuário.
    tipo_solicitacao: 'avaliacao_livro' ou 'lista_recomendacoes'.
    """
    instrucao = "Você é um Curador de Recomendações Literárias amigável e perspicaz.\n"

    if tipo_solicitacao == 'avaliacao_livro':
        instrucao += f"""
        Com base no "Relatório do Detetive Literário" sobre o livro "{livro_original_ou_tema}", sua tarefa é apresentar uma conclusão amigável para o usuário.
        Resuma os pontos chave do relatório, destacando se o livro parece ser uma boa escolha para um leitor em geral ou para alguém com um perfil específico (se fornecido).
        Seja direto e útil. O objetivo é ajudar o usuário a decidir se deve ou não ler este livro.
        Se o perfil do leitor foi fornecido, personalize a recomendação diretamente para ele.
        Finalize com uma frase como: "Com base nisso, este livro parece ser [muito recomendado / recomendado com ressalvas / talvez não seja o ideal para você agora]..."
        """
        entrada_agente_texto = f"Dados para recomendação (Perfil do Livro):\n{dados_entrada}\n\nLivro Original Avaliado: {livro_original_ou_tema}"
    elif tipo_solicitacao == 'lista_recomendacoes':
        instrucao += f"""
        Com base na lista de "Livros Similares" fornecida pelo Agente Conector de Histórias, referente a "{livro_original_ou_tema}", sua tarefa é apresentar essa lista de forma atraente e convincente ao usuário.
        Para cada livro recomendado:
        - Reafirme o Título e Autor.
        - Expanda um pouco a justificativa da similaridade de forma mais convidativa.
        - Adicione uma pequena frase sobre o que o leitor pode esperar do livro (10-15 palavras).
        Se o perfil do leitor foi fornecido, mencione sutilmente por que essas recomendações podem agradá-lo.
        Organize como uma lista numerada. Comece com uma frase como: "Com base no seu interesse em '{livro_original_ou_tema}', aqui estão algumas sugestões que você pode gostar:"
        """
        entrada_agente_texto = f"Dados para recomendação (Lista de Similares):\n{dados_entrada}\n\nReferência Original: {livro_original_ou_tema}"
    else:
        return "Tipo de solicitação inválido para o Agente Curador de Recomendações."

    if perfil_do_leitor:
        instrucao += f"\n\nPerfil do leitor para consideração: {perfil_do_leitor}"


    curador = Agent(
        name="agente_curador_de_recomendacoes",
        model=MODEL_ID,
        instruction=instrucao,
        description="Agente que apresenta recomendações literárias finais de forma clara e útil."
        # Geralmente não precisa de 'tools' aqui, pois apenas formata e apresenta dados.
    )
    recomendacao_final = call_agent(curador, entrada_agente_texto)
    return recomendacao_final

# Etapa 5: Orquestração Interativa dos Agentes

print("📚 Bem-vindo ao Sistema de Descoberta e Recomendação Literária! 📚")
print("------------------------------------------------------------------\n")

# Opcional: Coletar um breve perfil do leitor
perfil_leitor = input("✨ Para recomendações mais personalizadas, descreva brevemente seus gostos literários (gêneros preferidos, autores, o que busca em um livro) ou deixe em branco: ").strip()
if perfil_leitor:
    print(f"Entendido! Consideraremos seu perfil: {perfil_leitor}\n")
else:
    print("Ok, seguiremos com recomendações mais gerais.\n")


print("Como posso te ajudar hoje?")
print("1. Quero saber mais sobre um livro específico (para decidir se leio).")
print("2. Quero recomendações baseadas em um livro que já gostei.")
print("3. Quero recomendações baseadas em um tema ou gênero de interesse.")

while True:
    try:
        escolha = int(input("Digite o número da sua opção: "))
        if escolha in [1, 2, 3]:
            break
        else:
            print("Opção inválida. Por favor, digite 1, 2 ou 3.")
    except ValueError:
        print("Entrada inválida. Por favor, digite um número.")

print("\n--------------------------------------------------------------")

if escolha == 1:
    livro_interesse = input("Qual livro você gostaria de investigar? Digite o título: ").strip()
    if not livro_interesse:
        print("🚫 Você não digitou um título de livro.")
    else:
        print(f"\n🕵️ Agente Detetive Literário investigando '{livro_interesse}'...")
        perfil_livro = agente_detetive_literario(livro_interesse, perfil_leitor)
        display(Markdown("--- 🔎 **Relatório do Detetive:** ---"))
        display(to_markdown(perfil_livro))

        print(f"\n큐레이터 Agente Curador de Recomendações preparando o veredito...")
        veredito_final = agente_curador_de_recomendacoes(perfil_livro, 'avaliacao_livro', livro_interesse, perfil_leitor)
        display(Markdown("\n--- 💡 **Veredito do Curador:** ---"))
        display(to_markdown(veredito_final))

elif escolha == 2:
    livro_base = input("Qual livro você amou e gostaria de encontrar similares? Digite o título: ").strip()
    if not livro_base:
        print("🚫 Você não digitou um título de livro.")
    else:
        print(f"\n🔗 Agente Conector de Histórias buscando similares a '{livro_base}'...")
        livros_similares = agente_conector_de_historias(livro_base, 'livro_base', perfil_leitor)
        display(Markdown(f"--- 📚 **Livros Similares a '{livro_base}' Encontrados:** ---"))
        display(to_markdown(livros_similares))

        print(f"\n큐레이터 Agente Curador de Recomendações montando sua lista personalizada...")
        recomendacoes_finais = agente_curador_de_recomendacoes(livros_similares, 'lista_recomendacoes', livro_base, perfil_leitor)
        display(Markdown("\n--- ✨ **Suas Recomendações Personalizadas:** ---"))
        display(to_markdown(recomendacoes_finais))

elif escolha == 3:
    tema_interesse = input("Sobre qual tema, gênero ou tipo de história você gostaria de recomendações? ").strip()
    if not tema_interesse:
        print("🚫 Você não digitou um tema.")
    else:
        print(f"\n🔗 Agente Conector de Histórias explorando o tema '{tema_interesse}'...")
        livros_sobre_tema = agente_conector_de_historias(tema_interesse, 'tema_interesse', perfil_leitor)
        display(Markdown(f"--- 📚 **Livros sobre '{tema_interesse}' Encontrados:** ---"))
        display(to_markdown(livros_sobre_tema))

        print(f"\n큐레이터 Agente Curador de Recomendações montando sua lista temática...")
        recomendacoes_finais = agente_curador_de_recomendacoes(livros_sobre_tema, 'lista_recomendacoes', tema_interesse, perfil_leitor)
        display(Markdown("\n--- ✨ **Suas Recomendações Temáticas:** ---"))
        display(to_markdown(recomendacoes_finais))

print("\n--------------------------------------------------------------")
print("✨ Espero ter ajudado em sua jornada literária! ✨")

📚 Bem-vindo ao Sistema de Descoberta e Recomendação Literária! 📚
------------------------------------------------------------------

✨ Para recomendações mais personalizadas, descreva brevemente seus gostos literários (gêneros preferidos, autores, o que busca em um livro) ou deixe em branco: Gosto do livro O Ceifador, Como Tigres na Neve, Gatos Alados, Clarice Lispector, Literatura japonesa
Entendido! Consideraremos seu perfil: Gosto do livro O Ceifador, Como Tigres na Neve, Gatos Alados, Clarice Lispector, Literatura japonesa

Como posso te ajudar hoje?
1. Quero saber mais sobre um livro específico (para decidir se leio).
2. Quero recomendações baseadas em um livro que já gostei.
3. Quero recomendações baseadas em um tema ou gênero de interesse.
Digite o número da sua opção: 1

--------------------------------------------------------------
Qual livro você gostaria de investigar? Digite o título: O Mundo de Sofia

🕵️ Agente Detetive Literário investigando 'O Mundo de Sofia'...


--- 🔎 **Relatório do Detetive:** ---

> Certo, Detetive Literário em ação! Investigarei "O Mundo de Sofia" para criar um perfil conciso do livro.
> 
> 
> ## Relatório de Investigação Literária: "O Mundo de Sofia"
> 
> ### 1. Sinopse Breve:
> "O Mundo de Sofia" conta a história de Sofia Amundsen, uma adolescente norueguesa que, às vésperas de seu aniversário de 15 anos, começa a receber cartas misteriosas com perguntas filosóficas intrigantes, como "Quem é você?" e "De onde vem o mundo?". Essas mensagens a levam a um curso intensivo de filosofia ministrado por um professor enigmático, Alberto Knox. Paralelamente, Sofia encontra cartões postais destinados a uma garota chamada Hilde, enviados do Líbano, o que a envolve em um mistério ainda maior. A trama combina a jornada de autoconhecimento de Sofia com uma exploração acessível da história da filosofia ocidental, desde os pré-socráticos até os existencialistas.
> 
> ### 2. Temas Principais:
> 
> *   **História da Filosofia:** O livro percorre a trajetória do pensamento filosófico ocidental, apresentando as ideias de diversos filósofos de forma didática e envolvente.
> *   **Existencialismo:** Questões sobre a existência, a identidade, o sentido da vida e o papel do ser humano no universo são exploradas ao longo da narrativa.
> *   **Realidade vs. Ficção:** A trama questiona a natureza da realidade e a relação entre o mundo real e as construções narrativas, culminando em uma reviravolta surpreendente sobre a própria história de Sofia.
> *   **Amadurecimento:** A jornada de Sofia é também uma história de amadurecimento, na qual ela busca compreender o mundo e a si mesma, questionando valores e crenças estabelecidas.
> 
> ### 3. Estilo de Escrita e Tom:
> 
> O estilo de escrita de Jostein Gaarder é acessível, envolvente e didático. Ele combina elementos de romance juvenil com explicações claras e concisas sobre conceitos filosóficos complexos. O tom é instigante, misterioso e reflexivo, convidando o leitor a questionar suas próprias certezas e a explorar novas perspectivas. Há um equilíbrio entre a narrativa ficcional e a exposição filosófica, tornando a leitura agradável e informativa.
> 
> ### 4. Sentimento Geral / Vibe:
> 
> O livro evoca uma sensação de **curiosidade intelectual** e **despertar filosófico**. Ele inspira o leitor a refletir sobre o mundo e a si mesmo, a questionar o status quo e a buscar um entendimento mais profundo da existência. O mistério que envolve a trama adiciona um elemento de **suspense** e **intrigante**, mantendo o leitor engajado e ansioso para descobrir as respostas. Há também um toque de **maravilhosamente estranho**, especialmente à medida que a linha entre a realidade e a ficção se torna mais tênue.
> 
> ### 5. Público-Alvo Comum / Para Quem é Recomendado:
> 
> "O Mundo de Sofia" é recomendado para:
> 
> *   **Jovens leitores** interessados em filosofia ou em busca de um livro que os faça pensar.
> *   **Adultos** que desejam ter uma introdução acessível à história da filosofia.
> *   **Professores e estudantes** de filosofia que buscam uma obra didática e envolvente para complementar seus estudos.
> *   **Leitores** que apreciam romances com elementos de mistério, suspense e reflexão existencial.
> 
> ### 6. Pontos Fortes Comuns:
> 
> *   **Acessibilidade:** O livro torna a filosofia acessível a um público amplo, sem simplificar demais os conceitos.
> *   **Envolvimento:** A trama envolvente e os personagens cativantes prendem a atenção do leitor do início ao fim.
> *   **Didatismo:** A forma como a história da filosofia é apresentada facilita a compreensão e o aprendizado.
> *   **Reflexão:** O livro estimula o pensamento crítico e a reflexão sobre questões existenciais importantes.
> *   **Originalidade:** A combinação de romance juvenil com história da filosofia é inovadora e surpreendente.
> 
> ### 7. Possíveis Pontos Fracos ou Advertências:
> 
> *   **Complexidade:** Alguns leitores podem achar a quantidade de informações filosóficas um pouco densa, especialmente se não tiverem familiaridade com o assunto.
> *   **Final:** O final do livro pode ser considerado confuso ou insatisfatório por alguns leitores, devido à sua natureza metaficcional e às reviravoltas na trama.
> *   **Ritmo:** Em alguns momentos, a narrativa pode se tornar um pouco lenta, especialmente nas partes em que a exposição filosófica é mais extensa.
> 
> **Considerações adicionais para o perfil do leitor:**
> 
> Considerando que o leitor aprecia obras como "O Ceifador", "Como Tigres na Neve", "Gatos Alados", Clarice Lispector e literatura japonesa, "O Mundo de Sofia" pode ser uma boa escolha, especialmente se o interesse for por obras que misturam elementos fantásticos, reflexões profundas e personagens em busca de autoconhecimento. A escrita de Clarice Lispector tem algumas coisas em comum com o livro, a necessidade de sempre estar questionando a realidade e a existência. Se o leitor estiver aberto a uma abordagem mais didática da filosofia, "O Mundo de Sofia" pode ser uma experiência enriquecedora.
> 



큐레이터 Agente Curador de Recomendações preparando o veredito...



--- 💡 **Veredito do Curador:** ---

> Com base no relatório do Detetive Literário, "O Mundo de Sofia" parece ser uma escolha interessante para você. O livro combina elementos de mistério e fantasia com reflexões filosóficas, o que se alinha com seu gosto por obras como "O Ceifador", "Como Tigres na Neve" e "Gatos Alados". Além disso, a profundidade e o questionamento existencial presentes na escrita de Clarice Lispector também encontram eco neste livro.
> 
> A abordagem didática da filosofia pode ser um diferencial, oferecendo uma nova perspectiva sobre temas que você já aprecia. No entanto, esteja ciente de que o livro pode ser um pouco denso em alguns momentos, devido à quantidade de informações filosóficas.
> 
> Com base nisso, este livro parece ser recomendado com ressalvas para você.



--------------------------------------------------------------
✨ Espero ter ajudado em sua jornada literária! ✨
